In [1]:
# Install all the required packages (borrowed from openWakeWord's automatic training notebook)
running_on_colab = False
if 'google.colab' in str(get_ipython()):
  print('Running on CoLab')
  from google.colab import drive
  drive.mount('/content/drive')
  running_on_colab = True
else:
  print('Not running on CoLab')
  restore_dataset = False
  restore_features = False

Not running on CoLab


In [2]:

if running_on_colab:
  dataset_filename = "datasets_20240423_175513.tar"
  features_filename = "training_features_20240421_155624.tar"
  if restore_dataset:
    !cp /content/drive/MyDrive/ColabNotebooks/VoiceAssistant/microWakeWord/{dataset_filename}  .
    !tar -xvf ./{dataset_filename}
  if restore_features:
    !cp /content/drive/MyDrive/ColabNotebooks/VoiceAssistant/microWakeWord/{features_filename}  .
    !tar -xvf ./{features_filename}

In [ ]:
%pip install -q torch
%pip install -q torchaudio
%pip install -q scipy
%pip install -q tqdm
%pip install -q jupyter
%pip install -q ipywidgets
%pip install -q mutagen
%pip install -q torchinfo
%pip install -q torchmetrics
%pip install -q speechbrain
%pip install -q audiomentations
%pip install -q torch-audiomentations
%pip install -q acoustics
%pip install -q pronouncing
%pip install -q datasets
%pip install -q deep-phonemizer
%pip install -q piper-phonemize
%pip install -q webrtcvad
%pip install -q datasets
%pip install -q mmap_ninja
%pip install -q gradio
%pip install -q tensorflow==2.15.0
%pip install -q keras==2.15.0
%pip install -q sounddevice
%pip install -q tflite_micro
#%pip install cloud-tpu-client

In [3]:
# Dataset generation
import os
import sys

# if not os.path.isdir("./piper-sample-generator"):
#     !git clone https://github.com/rhasspy/piper-sample-generator
#     !wget -O piper-sample-generator/models/en_US-libritts_r-medium.pt 'https://github.com/rhasspy/piper-sample-generator/releases/download/v2.0.0/en_US-libritts_r-medium.pt'


if not os.path.isdir("./openWakeWord"):
    !git clone https://github.com/dscripka/openWakeWord

cwd = os.getcwd()
# piper_path = cwd+"/piper-sample-generator/"
# if piper_path not in sys.path:
#     sys.path.insert(0, piper_path)

oww_path = cwd + "/openWakeWord/openwakeword/"
if  oww_path not in sys.path:
    sys.path.insert(0, oww_path)

print("Updated sys.path is ")
print("\n".join(sys.path))
print("Please restart the kernel (Kernel -> Restart Kernel) and run the next cell.")

Updated sys.path is 
/home/lior/Custom-MicroWakeWord-Generator/openWakeWord/openwakeword/
/home/lior/Custom-MicroWakeWord-Generator
/opt/conda/envs/google_kws/lib/python311.zip
/opt/conda/envs/google_kws/lib/python3.11
/opt/conda/envs/google_kws/lib/python3.11/lib-dynload

/opt/conda/envs/google_kws/lib/python3.11/site-packages
Please restart the kernel (Kernel -> Restart Kernel) and run the next cell.


In [4]:
import sys
import os
from pathlib import Path
from tqdm import tqdm
from mmap_ninja.ragged import RaggedMmap
import os
import uuid
import logging
import datasets
import scipy
import numpy as np
import tqdm
import torch
import yaml
import logging
# from generate_samples import generate_samples
from data import generate_adversarial_texts


torchvision is not available - cannot save figures


In [5]:
# realod the json config saved in teh dataset geenration notebook
import json
# Open the file and load the JSON
with open('dataset_config.json', 'r') as f:
    config = json.load(f)

# Now config is a dictionary with the same content as when it was saved
print(config)

config["samples_output_dir"] = os.path.abspath(config["samples_output_dir"])

if not os.path.exists(config["samples_output_dir"]):
    os.mkdir(config["samples_output_dir"])

models_samples_directory = os.path.join(config["samples_output_dir"], config["model_name"])
if not os.path.exists(models_samples_directory):
    os.mkdir(models_samples_directory)

positive_train_output_dir = os.path.join(models_samples_directory, "positive_train")
positive_test_output_dir = os.path.join(models_samples_directory, "positive_test")
positive_validation_output_dir = os.path.join(models_samples_directory, "positive_validation")
negative_train_output_dir = os.path.join(models_samples_directory, "negative_train")
negative_test_output_dir = os.path.join(models_samples_directory, "negative_test")
negative_validation_output_dir = os.path.join(models_samples_directory, "negative_validation")



{'target_phrase': 'alexha', 'target_phrase_as_written': 'alexha', 'model_name': 'alexa', 'custom_negative_phrases': ['ali', 'aliba', 'yooba', 'liba', 'tessa', 'alex', 'exa', 'ale'], 'samples_output_dir': '/home/lior/Custom-MicroWakeWord-Generator/datasets/generated_samples', 'features_output_dir': 'training_features', 'rirs_dir': 'datasets/BUT_ReverbDB_rel_19_06_RIR-Only', 'rirs_glob': '**/RIR/*.wav', 'audioset_clips_dir': 'datasets/audioset_16k', 'fma_clips_dir': 'datasets/fma', 'fsd_clips_dir': 'datasets/fsd', 'n_samples': 250000, 'n_samples_val': 20000, 'tts_batch_size': 20, 'augment_batch_size': 16, 'clip_duration_ms': 1430, 'spectrogram_duration_ms': 1490, 'sample_rate_hz': 16000, 'end_jitter_ms': 60}


In [12]:
# get the preprocessor tf lite mode
if os.path.exists('audio_preprocessor_int8.tflite') == False:
    !wget https://github.com/tensorflow/tflite-micro/raw/main/tensorflow/lite/micro/examples/micro_speech/models/audio_preprocessor_int8.tflite

--2024-04-29 12:54:29--  https://github.com/tensorflow/tflite-micro/raw/main/tensorflow/lite/micro/examples/micro_speech/models/audio_preprocessor_int8.tflite
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/tensorflow/tflite-micro/main/tensorflow/lite/micro/examples/micro_speech/models/audio_preprocessor_int8.tflite [following]
--2024-04-29 12:54:29--  https://raw.githubusercontent.com/tensorflow/tflite-micro/main/tensorflow/lite/micro/examples/micro_speech/models/audio_preprocessor_int8.tflite
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8772 (8.6K) [application/octet-stream]
Saving to: 'aud

In [7]:
# Augment generated samples by adding background noise and applying room impulse responses
from data import augment_clips, filter_audio_paths

config["rirs_dir"] = os.path.abspath(config["rirs_dir"])
config["audioset_clips_dir"] = os.path.abspath(config["audioset_clips_dir"])
config["fma_clips_dir"] = os.path.abspath(config["fma_clips_dir"])
config["fsd_clips_dir"] = os.path.abspath(config["fsd_clips_dir"])
config["features_output_dir"] = os.path.abspath(config["features_output_dir"])
config["audio_samples_per_clip"] = int((config["spectrogram_duration_ms"])*config["sample_rate_hz"]/1000) # ms * herz *1/(1000ms) = # of samples

print("audio_samples_per_clip: ", config["audio_samples_per_clip"])

if not os.path.exists(config["features_output_dir"]):
    os.mkdir(config["features_output_dir"])


audio_samples_per_clip:  23840


In [8]:

max_duration_sec = config["clip_duration_ms"]/1000.0
print("filtering with max_duration_sec: ", max_duration_sec, " sec")

spectrogram_duration_sec = config["spectrogram_duration_ms"]/1000.0
jitter_s = config["end_jitter_ms"]/1000.0

positive_tts_clips_train, durations = filter_audio_paths([positive_train_output_dir],min_length_secs=0.0, max_length_secs=max_duration_sec, duration_method = "size")
positive_tts_clips_test, durations = filter_audio_paths([positive_test_output_dir],min_length_secs=0.0, max_length_secs=max_duration_sec, duration_method = "size")
positive_tts_clips_validation, durations = filter_audio_paths([positive_validation_output_dir],min_length_secs=0.0, max_length_secs=max_duration_sec, duration_method = "size")

negative_tts_clips_train, durations = filter_audio_paths([negative_train_output_dir],min_length_secs=0.0, max_length_secs=max_duration_sec, duration_method = "size")
negative_tts_clips_test, durations = filter_audio_paths([negative_test_output_dir],min_length_secs=0.0, max_length_secs=max_duration_sec, duration_method = "size")
negative_tts_clips_validation, durations = filter_audio_paths([negative_validation_output_dir],min_length_secs=0.0, max_length_secs=max_duration_sec, duration_method = "size")

rir_paths = [str(i) for i in Path(config["rirs_dir"]).glob(config["rirs_glob"])]


filtering with max_duration_sec:  1.43  sec


300000it [00:01, 237471.82it/s]
20000it [00:00, 252475.00it/s]
20000it [00:00, 255032.58it/s]
252296it [00:01, 247808.76it/s]
20000it [00:00, 247428.21it/s]
20000it [00:00, 253051.54it/s]


In [9]:
if restore_features == False:
  !rm -rf ./training_features

In [45]:
# the following swithc uses the tflite preprocessor (same one used in the embedded device) as opposed of using
# the microfrontend s/w based one
tflite_prep = True

import tensorflow as tf
from tensorflow.lite.experimental.microfrontend.python.ops import audio_microfrontend_op as frontend_op
from tflite_micro.python.tflite_micro import runtime

preprocessor_model = runtime.Interpreter.from_file("./audio_preprocessor_int8.tflite")
input_details = preprocessor_model.get_input_details(0)
output_details = preprocessor_model.get_output_details(0)
preprocessor_model.print_allocations()

class VectorSplitter:
    def __init__(self, chunk_size=480):
        self.chunk_size = chunk_size
        self.remainder = np.zeros(160)

    def split_into_chunks(self, vector):
        chunks = []
        #print(f"Splitting vector of size {len(vector)}, stored remainder is of size {len(self.remainder)}")
        vector = np.concatenate((self.remainder, vector))
        #print(f"Concatenated vector of size {len(vector)}")
        i = 0
        while i + self.chunk_size <= len(vector):
        #for i in range(0, len(vector), self.chunk_size):
            chunk = vector[i:i + self.chunk_size]
            chunks.append(chunk)
            #print("append chunk ", chunk.shape, " i:", i)
            #self.remainder = vector[-160:]
            #print("Remainder:", self.remainder.shape)
            i += 320
        #if i < len(vector):
        self.remainder = vector[i:]
        #print("end of vector Remainder:", self.remainder.shape)
        return chunks

splitter = VectorSplitter()
def get_features(input):
    if len(input) != 480:
        raise ValueError("Input must be of size 480")
        return
    preprocessor_model.set_input(input.reshape([1,480]).astype(np.int16), 0)
    preprocessor_model.invoke()
    return preprocessor_model.get_output(0)

# this generates the MEL spectrogram features for a given clip
def generate_features_for_clip(clip):

    if tflite_prep == True:
        chunks =  splitter.split_into_chunks(clip)
        matrix = np.array([get_features(chunk) for chunk in chunks])
        return matrix
    else:
        micro_frontend = frontend_op.audio_microfrontend(
            tf.convert_to_tensor(clip),
            sample_rate=16000,
            window_size=30,
            window_step=20,
            num_channels=40,
            upper_band_limit=7500,
            lower_band_limit=125,
            enable_pcan=True,
            min_signal_remaining=0.05,
            out_scale=1,
            out_type=tf.float32)
        output = tf.multiply(micro_frontend, 0.0390625)
        return output.numpy()

def clip_features_generator(generator):
    for data in generator:
        for clip in data:
            yield generate_features_for_clip(clip)

Number of resource variables the model uses =  0


[RecordingMicroAllocator] Arena allocation total 11800 bytes
[RecordingMicroAllocator] Arena allocation head 3040 bytes
[RecordingMicroAllocator] Arena allocation tail 8760 bytes
[RecordingMicroAllocator] 'TfLiteEvalTensor data' used 1032 bytes with alignment overhead (requested 1032 bytes for 43 allocations)
[RecordingMicroAllocator] 'Persistent TfLiteTensor data' used 224 bytes with alignment overhead (requested 224 bytes for 2 tensors)
[RecordingMicroAllocator] 'Persistent buffer data' used 4988 bytes with alignment overhead (requested 4872 bytes for 19 allocations)
[RecordingMicroAllocator] 'NodeAndRegistration struct' used 1936 bytes with alignment overhead (requested 1936 bytes for 22 NodeAndRegistration structs)


In [14]:
###################################
# Generate features for TTS samples
###################################
for i in range(0,1):
    positive_train_generator = augment_clips(positive_tts_clips_train,
                        total_length=config["audio_samples_per_clip"],
                        batch_size=config["augment_batch_size"],
                        background_clip_paths=[config["audioset_clips_dir"], config["fma_clips_dir"], config["fsd_clips_dir"]],
                        RIR_paths=rir_paths)
    positive_test_generator = augment_clips(positive_tts_clips_test,
                        total_length=config["audio_samples_per_clip"],
                        batch_size=config["augment_batch_size"],
                        background_clip_paths=[config["audioset_clips_dir"], config["fma_clips_dir"], config["fsd_clips_dir"]],
                        RIR_paths=rir_paths)
    positive_validation_generator = augment_clips(positive_tts_clips_validation,
                        total_length=config["audio_samples_per_clip"],
                        batch_size=config["augment_batch_size"],
                        background_clip_paths=[config["audioset_clips_dir"], config["fma_clips_dir"], config["fsd_clips_dir"]],
                        RIR_paths=rir_paths)

    negative_train_generator = augment_clips(negative_tts_clips_train,
                        total_length=config["audio_samples_per_clip"],
                        batch_size=config["augment_batch_size"],
                        background_clip_paths=[config["audioset_clips_dir"], config["fma_clips_dir"], config["fsd_clips_dir"]],
                        RIR_paths=rir_paths)
    negative_test_generator = augment_clips(negative_tts_clips_test,
                        total_length=config["audio_samples_per_clip"],
                        batch_size=config["augment_batch_size"],
                        background_clip_paths=[config["audioset_clips_dir"], config["fma_clips_dir"], config["fsd_clips_dir"]],
                        RIR_paths=rir_paths)
    negative_validation_generator = augment_clips(negative_tts_clips_validation,
                        total_length=config["audio_samples_per_clip"],
                        batch_size=config["augment_batch_size"],
                        background_clip_paths=[config["audioset_clips_dir"], config["fma_clips_dir"], config["fsd_clips_dir"]],
                        RIR_paths=rir_paths)

    print(positive_train_generator)


    dataset_name = "positive_tts_samples"
    pos_dir_fname = "wakeword"
    augmented_positive_test_directory = os.path.join(config["features_output_dir"] + "/" + dataset_name + "/testing", pos_dir_fname)
    augmented_positive_validation_directory = os.path.join(config["features_output_dir"] + "/" + dataset_name + "/validation", pos_dir_fname)
    augmented_positive_train_directory = os.path.join(config["features_output_dir"] + "/" + dataset_name + "/training", pos_dir_fname)

    dataset_name = "negative_tts_samples"
    neg_dir_fname = "unknown"
    augmented_negative_test_directory = os.path.join(config["features_output_dir"] + "/" + dataset_name + "/testing", neg_dir_fname)
    augmented_negative_validation_directory = os.path.join(config["features_output_dir"] + "/" + dataset_name + "/validation", neg_dir_fname)
    augmented_negative_train_directory = os.path.join(config["features_output_dir"] + "/" + dataset_name + "/training", neg_dir_fname)

    generator_outputs = [
                         [positive_train_generator, augmented_positive_train_directory, len(positive_tts_clips_train)],
                         [positive_test_generator, augmented_positive_test_directory,len(positive_tts_clips_test)],
                         [positive_validation_generator, augmented_positive_validation_directory, len(positive_tts_clips_validation)],
                         [negative_train_generator, augmented_negative_train_directory, len(negative_tts_clips_train)],
                         [negative_test_generator, augmented_negative_test_directory, len(negative_tts_clips_test)],
                         [negative_validation_generator, augmented_negative_validation_directory, len(negative_tts_clips_validation)]
                        ]


    for [generator, output_directory, n_total] in generator_outputs:

        output_directory = output_directory + '_batch_'+str(i)+'_mmap'
        print("Generating ", n_total, " features into ", output_directory)
        if not os.path.exists(output_directory):
            os.makedirs(output_directory, exist_ok=True)

        RaggedMmap.from_generator(out_dir=output_directory,sample_generator=clip_features_generator(generator),batch_size=1024,verbose=True)




<generator object augment_clips at 0x7f79c2dd8a00>
Generating  299981  features into  /home/lior/Custom-MicroWakeWord-Generator/training_features/positive_tts_samples/training/wakeword_batch_0_mmap


0it [00:00, ?it/s]

Generating  19999  features into  /home/lior/Custom-MicroWakeWord-Generator/training_features/positive_tts_samples/testing/wakeword_batch_0_mmap


0it [00:00, ?it/s]

Generating  19998  features into  /home/lior/Custom-MicroWakeWord-Generator/training_features/positive_tts_samples/validation/wakeword_batch_0_mmap


...

0it [00:00, ?it/s]

Generating  19886  features into  /home/lior/Custom-MicroWakeWord-Generator/training_features/negative_tts_samples/testing/unknown_batch_0_mmap


0it [00:00, ?it/s]

Generating  19898  features into  /home/lior/Custom-MicroWakeWord-Generator/training_features/negative_tts_samples/validation/unknown_batch_0_mmap


0it [00:00, ?it/s]

In [48]:

### NOT FUNCTIONAL #### contains RIR only 


# Generates features for background silence datasets
# Input files do not need to be pre-converted to wav
# Test and validations sets are truncated to the clip duration for consistent testing
# Training sets convert the entire clip; the training process randomly truncates it each time used

import datasets
import sys
import numpy as np
from pathlib import Path
from tqdm import tqdm
from mmap_ninja.ragged import RaggedMmap
from data import truncate_clip

logging.getLogger('numba').setLevel(logging.WARNING)

# from http://merlin.fit.vutbr.cz/ReverbDB/BUT_ReverbDB_rel_19_06_RIR-Only.tgz

path_to_audio_dataset = "datasets/BUT_ReverbDB_rel_19_06_RIR-Only"
audio_dataset_glob = "**/silence/*.wav"
dataset_name = "but_reverdb_silence"

audioset_dataset = datasets.Dataset.from_dict({"audio": [str(i) for i in Path(path_to_audio_dataset).glob(audio_dataset_glob)]})
audioset_dataset = audioset_dataset.cast_column("audio", datasets.Audio(sampling_rate=16000))
print("found ", len(audioset_dataset), " audio files")
train_testvalid = audioset_dataset.train_test_split(test_size=0.1)

test_validate = train_testvalid['test'].train_test_split(test_size=0.5)

def features_generator(set):
    if set == 'train':
        for row in train_testvalid['train']:
            if len(row['audio']['array']) < config["audio_samples_per_clip"]:  # ensure clip has at least as many samples as needed
                continue
            yield generate_features_for_clip((row['audio']['array']*32767).astype(np.int16))
    elif set == 'test':
        for row in test_validate['test']:
            if len(row['audio']['array']) < config["audio_samples_per_clip"]:  # ensure clip has at least as many samples as needed
                continue

            # Truncate for a consistent test set
            truncated = truncate_clip((row['audio']['array']*32767).astype(np.int16), config["audio_samples_per_clip"], "random")
            yield generate_features_for_clip(truncated)
    elif set == 'validate':
        for row in test_validate['train']:
            if len(row['audio']['array']) < config["audio_samples_per_clip"]: # ensure clip has at least as many samples as needed
                continue

            # Truncate for a consistent validation set
            truncated = truncate_clip((row['audio']['array']*32767).astype(np.int16), config["audio_samples_per_clip"], "random")
            yield generate_features_for_clip(truncated)


test_dir_fname = dataset_name + '_test_' + str(config["audio_samples_per_clip"]) + 'ms_mmap'
validation_dir_fname = dataset_name + '_validation_' + str(config["audio_samples_per_clip"]) + 'ms_mmap'
train_dir_fname = dataset_name + '_training_' + '_mmap'


test_output_dir = os.path.join(config["features_output_dir"] + "/" + dataset_name + "/testing", test_dir_fname)
validation_output_dir = os.path.join(config["features_output_dir"] + "/" + dataset_name + "/validation", validation_dir_fname)
train_output_dir = os.path.join(config["features_output_dir"] + "/" + dataset_name + "/training", train_dir_fname)

if not os.path.exists(test_output_dir):
    os.makedirs(test_output_dir)
if not os.path.exists(validation_output_dir):
    os.makedirs(validation_output_dir)
if not os.path.exists(train_output_dir):
    os.makedirs(train_output_dir)

print("test_output_dir:", test_output_dir)
print("validation_output_dir:", validation_output_dir)
print("train_output_dir:", train_output_dir)

print("Generating features for test set")
RaggedMmap.from_generator(out_dir=test_output_dir,
                          sample_generator=features_generator('test'),
                          batch_size=1024,
                          verbose=True)
print("Generating features for validation set")
RaggedMmap.from_generator(out_dir=validation_output_dir,
                          sample_generator=features_generator('validate'),
                          batch_size=1024,
                          verbose=True)
print("Generating features for training set")
RaggedMmap.from_generator(out_dir=train_output_dir,
                          sample_generator=features_generator('train'),
                          batch_size=1024,
                          verbose=True)


found  0  audio files
test_output_dir: /home/lior/Custom-MicroWakeWord-Generator/training_features/but_reverdb_silence/testing/but_reverdb_silence_test_23840ms_mmap
validation_output_dir: /home/lior/Custom-MicroWakeWord-Generator/training_features/but_reverdb_silence/validation/but_reverdb_silence_validation_23840ms_mmap
train_output_dir: /home/lior/Custom-MicroWakeWord-Generator/training_features/but_reverdb_silence/training/but_reverdb_silence_training__mmap
Generating features for test set


0it [00:00, ?it/s]

Generating features for validation set


0it [00:00, ?it/s]

Generating features for training set


0it [00:00, ?it/s]

In [16]:
###################################
# download background noise
# from WHAM from http://wham.whisper.ai/
###################################

path_to_audio_dataset = "datasets/wham_noise"
audio_dataset_glob = "*.wav"
dataset_name = "wham_noise"

audioset_dataset = datasets.Dataset.from_dict({"audio": [str(i) for i in Path(path_to_audio_dataset).glob(audio_dataset_glob)]})
audioset_dataset = audioset_dataset.cast_column("audio", datasets.Audio(sampling_rate=16000))

train_testvalid = audioset_dataset.train_test_split(test_size=0.1)

test_validate = train_testvalid['test'].train_test_split(test_size=0.5)

def features_generator(set):
    if set == 'train':
        for row in train_testvalid['train']:
            if len(row['audio']['array']) < config["audio_samples_per_clip"]:  # ensure clip has at least as many samples as needed
                continue
            yield generate_features_for_clip((row['audio']['array']*32767).astype(np.int16))
    elif set == 'test':
        for row in test_validate['test']:
            if len(row['audio']['array']) < config["audio_samples_per_clip"]:  # ensure clip has at least as many samples as needed
                continue

            # Truncate for a consistent test set
            truncated = truncate_clip((row['audio']['array']*32767).astype(np.int16), config["audio_samples_per_clip"], "random")
            yield generate_features_for_clip(truncated)
    elif set == 'validate':
        for row in test_validate['train']:
            if len(row['audio']['array']) < config["audio_samples_per_clip"]: # ensure clip has at least as many samples as needed
                continue

            # Truncate for a consistent validation set
            truncated = truncate_clip((row['audio']['array']*32767).astype(np.int16), config["audio_samples_per_clip"], "random")
            yield generate_features_for_clip(truncated)


test_dir_fname = dataset_name + '_test_' + str(config["audio_samples_per_clip"]) + 'ms_mmap'
validation_dir_fname = dataset_name + '_validation_' + str(config["audio_samples_per_clip"]) + 'ms_mmap'
train_dir_fname = dataset_name + '_training_' + '_mmap'


test_output_dir = os.path.join(config["features_output_dir"] + "/" + dataset_name + "/testing", test_dir_fname)
validation_output_dir = os.path.join(config["features_output_dir"] + "/" + dataset_name + "/validation", validation_dir_fname)
train_output_dir = os.path.join(config["features_output_dir"] + "/" + dataset_name + "/training", train_dir_fname)

if not os.path.exists(test_output_dir):
    os.makedirs(test_output_dir)
if not os.path.exists(validation_output_dir):
    os.makedirs(validation_output_dir)
if not os.path.exists(train_output_dir):
    os.makedirs(train_output_dir)

print("test_output_dir:", test_output_dir)
print("validation_output_dir:", validation_output_dir)
print("train_output_dir:", train_output_dir)

print("Generating features for test set")
RaggedMmap.from_generator(out_dir=test_output_dir,
                          sample_generator=features_generator('test'),
                          batch_size=1024,
                          verbose=True)
print("Generating features for validation set")
RaggedMmap.from_generator(out_dir=validation_output_dir,
                          sample_generator=features_generator('validate'),
                          batch_size=1024,
                          verbose=True)
print("Generating features for training set")
RaggedMmap.from_generator(out_dir=train_output_dir,
                          sample_generator=features_generator('train'),
                          batch_size=1024,
                          verbose=True)

print("Done generating features for background noise datasets - copying them to mWW source dataset directory")


test_output_dir: /home/lior/Custom-MicroWakeWord-Generator/training_features/wham_noise/testing/wham_noise_test_23840ms_mmap
validation_output_dir: /home/lior/Custom-MicroWakeWord-Generator/training_features/wham_noise/validation/wham_noise_validation_23840ms_mmap
train_output_dir: /home/lior/Custom-MicroWakeWord-Generator/training_features/wham_noise/training/wham_noise_training__mmap
Generating features for test set


0it [00:00, ?it/s]

Generating features for validation set


0it [00:00, ?it/s]

Generating features for training set


0it [00:00, ?it/s]

Done generating features for background noise datasets - copying them to mWW source dataset directory


In [17]:
###################################
# generate features for DIPCO noise
###################################

#
path_to_audio_dataset = "datasets/dipco"
audio_dataset_glob = "*.wav"
dataset_name = "dipco"

audioset_dataset = datasets.Dataset.from_dict({"audio": [str(i) for i in Path(path_to_audio_dataset).glob(audio_dataset_glob)]})
audioset_dataset = audioset_dataset.cast_column("audio", datasets.Audio(sampling_rate=16000))

train_testvalid = audioset_dataset.train_test_split(test_size=0.8)

test_validate = train_testvalid['test'].train_test_split(test_size=0.5)

def features_generator(set):
    if set == 'train':
        for row in train_testvalid['train']:
            if len(row['audio']['array']) < config["audio_samples_per_clip"]:  # ensure clip has at least as many samples as needed
                continue
            yield generate_features_for_clip((row['audio']['array']*32767).astype(np.int16))
    elif set == 'test':
        for row in test_validate['test']:
            if len(row['audio']['array']) < config["audio_samples_per_clip"]:  # ensure clip has at least as many samples as needed
                continue

            # # Truncate for a consistent test set
            # truncated = truncate_clip((row['audio']['array']*32767).astype(np.int16), config["audio_samples_per_clip"], "random")
            # yield generate_features_for_clip(truncated)
            yield generate_features_for_clip((row['audio']['array']*32767).astype(np.int16))
    elif set == 'validate':
        for row in test_validate['train']:
            if len(row['audio']['array']) < config["audio_samples_per_clip"]: # ensure clip has at least as many samples as needed
                continue

            # # Truncate for a consistent validation set
            # truncated = truncate_clip((row['audio']['array']*32767).astype(np.int16), config["audio_samples_per_clip"], "random")
            # yield generate_features_for_clip(truncated)
            yield generate_features_for_clip((row['audio']['array']*32767).astype(np.int16))


test_dir_fname = dataset_name + '_test_' + str(config["audio_samples_per_clip"]) + 'ms_mmap'
validation_dir_fname = dataset_name + '_validation_' + str(config["audio_samples_per_clip"]) + 'ms_mmap'
train_dir_fname = dataset_name + '_training_' + '_mmap'


test_output_dir = os.path.join(config["features_output_dir"] + "/" + dataset_name + "/testing_ambient", test_dir_fname)
validation_output_dir = os.path.join(config["features_output_dir"] + "/" + dataset_name + "/validation_ambient", validation_dir_fname)
train_output_dir = os.path.join(config["features_output_dir"] + "/" + dataset_name + "/training", train_dir_fname)

if not os.path.exists(test_output_dir):
    os.makedirs(test_output_dir)
if not os.path.exists(validation_output_dir):
    os.makedirs(validation_output_dir)
if not os.path.exists(train_output_dir):
    os.makedirs(train_output_dir)

print("test_output_dir:", test_output_dir)
print("validation_output_dir:", validation_output_dir)
print("train_output_dir:", train_output_dir)

print("Generating features for test set")
RaggedMmap.from_generator(out_dir=test_output_dir,
                          sample_generator=features_generator('test'),
                          batch_size=1024,
                          verbose=True)
print("Generating features for validation set")
RaggedMmap.from_generator(out_dir=validation_output_dir,
                          sample_generator=features_generator('validate'),
                          batch_size=1024,
                          verbose=True)
print("Generating features for training set")
RaggedMmap.from_generator(out_dir=train_output_dir,
                          sample_generator=features_generator('train'),
                          batch_size=1024,
                          verbose=True)

print("Done generating features for DIPCO noise datasets ")


test_output_dir: /home/lior/Custom-MicroWakeWord-Generator/training_features/dipco/testing_ambient/dipco_test_23840ms_mmap
validation_output_dir: /home/lior/Custom-MicroWakeWord-Generator/training_features/dipco/validation_ambient/dipco_validation_23840ms_mmap
train_output_dir: /home/lior/Custom-MicroWakeWord-Generator/training_features/dipco/training/dipco_training__mmap
Generating features for test set


0it [00:00, ?it/s]

Generating features for validation set


0it [00:00, ?it/s]

Generating features for training set


0it [00:00, ?it/s]

Done generating features for DIPCO noise datasets 


In [46]:
###################################
# generate features based on captured audio samples
###################################

max_duration_sec = 4
positive_captured_clips_train, durations = filter_audio_paths(['./datasets/captured_samples/positive_train/'],min_length_secs=0.0, max_length_secs=max_duration_sec, duration_method = "header")
positive_captured_clips_test, durations = filter_audio_paths(['./datasets/captured_samples/positive_test/'],min_length_secs=0.0, max_length_secs=max_duration_sec, duration_method = "header")
positive_captured_clips_validation, durations = filter_audio_paths(['./datasets/captured_samples/positive_validation'],min_length_secs=0.0, max_length_secs=max_duration_sec, duration_method = "header")

audio_samples_per_clip = config["audio_samples_per_clip"] #1.34*16000
for i in range(0,1):

    positive_train_generator = augment_clips(positive_captured_clips_train,
                        total_length=audio_samples_per_clip,
                        batch_size=config["augment_batch_size"],
                        background_clip_paths=[config["audioset_clips_dir"], config["fma_clips_dir"], config["fsd_clips_dir"]],
                        RIR_paths=rir_paths)
    positive_test_generator = augment_clips(positive_captured_clips_test,
                        total_length=audio_samples_per_clip,
                        batch_size=config["augment_batch_size"],
                        background_clip_paths=[config["audioset_clips_dir"], config["fma_clips_dir"], config["fsd_clips_dir"]],
                        RIR_paths=rir_paths)
    positive_validation_generator = augment_clips(positive_captured_clips_validation,
                        total_length=audio_samples_per_clip,
                        batch_size=config["augment_batch_size"],
                        background_clip_paths=[config["audioset_clips_dir"], config["fma_clips_dir"], config["fsd_clips_dir"]],
                        RIR_paths=rir_paths)



    print(positive_captured_clips_train)


    dataset_name = "positive_voice_samples"
    pos_dir_fname = "wakeword"
    augmented_positive_test_directory = os.path.join(config["features_output_dir"] + "/" + dataset_name + "/testing", pos_dir_fname)
    augmented_positive_validation_directory = os.path.join(config["features_output_dir"] + "/" + dataset_name + "/validation", pos_dir_fname)
    augmented_positive_train_directory = os.path.join(config["features_output_dir"] + "/" + dataset_name + "/training", pos_dir_fname)

    generator_outputs = [
                         [positive_train_generator, augmented_positive_train_directory, len(positive_captured_clips_train)],
                         [positive_test_generator, augmented_positive_test_directory,len(positive_captured_clips_test)],
                         [positive_validation_generator, augmented_positive_validation_directory, len(positive_captured_clips_validation)],
                        ]


    for [generator, output_directory, n_total] in generator_outputs:

        output_directory = output_directory + '_batch_'+str(i)+'_mmap'
        print("Generating ", n_total, " features into ", output_directory)
        if not os.path.exists(output_directory):
            os.makedirs(output_directory, exist_ok=True)

        RaggedMmap.from_generator(out_dir=output_directory,sample_generator=clip_features_generator(generator),batch_size=1,verbose=True)



16it [00:00, 44949.00it/s]
100%|██████████| 16/16 [00:00<00:00, 6355.01it/s]
4it [00:00, 11992.29it/s]
100%|██████████| 4/4 [00:00<00:00, 4153.80it/s]
4it [00:00, 11740.53it/s]
100%|██████████| 4/4 [00:00<00:00, 4356.59it/s]

['./datasets/captured_samples/positive_train/positive4058a60a3c714d0f98e9ebe3d1606164.wav', './datasets/captured_samples/positive_train/positive6430ab7269c34cd1b64e2bee610099d4.wav', './datasets/captured_samples/positive_train/positive7186a2b74f1c4bdfac48a47d7b51e1c8.wav', './datasets/captured_samples/positive_train/positive722e8929218e4707a66ea3a3de40ac67.wav', './datasets/captured_samples/positive_train/positive0da5fca5153246a8ae2e631f666622c8.wav', './datasets/captured_samples/positive_train/positive17bc63c2f2274366be22f956e0d84eff.wav', './datasets/captured_samples/positive_train/positivee23a131debd34c5faa7cc612349db651.wav', './datasets/captured_samples/positive_train/positivea41702dd97044a0cbe80d5f3f55a6473.wav', './datasets/captured_samples/positive_train/positive57044322985d4f3295cd19619fcc41e3.wav', './datasets/captured_samples/positive_train/positivef90b27b9fae6434c8deb523ade855ac0.wav', './datasets/captured_samples/positive_train/positivef9c8785ffede4d90ae7bb7213e98a6b8.wav'

0it [00:00, ?it/s]

Generating  4  features into  /home/lior/Custom-MicroWakeWord-Generator/training_features/positive_voice_samples/testing/wakeword_batch_0_mmap


0it [00:00, ?it/s]

Generating  4  features into  /home/lior/Custom-MicroWakeWord-Generator/training_features/positive_voice_samples/validation/wakeword_batch_0_mmap


0it [00:00, ?it/s]

In [42]:
###################################
# generate features based on COQUI'ed captured audio samples
###################################

max_duration_sec = 3
positive_cloned_clips_train, durations = filter_audio_paths(['./datasets/cloned_samples/positive_train/'],min_length_secs=0.0, max_length_secs=max_duration_sec, duration_method = "header")
positive_cloned_clips_test, durations = filter_audio_paths(['./datasets/cloned_samples/positive_test/'],min_length_secs=0.0, max_length_secs=max_duration_sec, duration_method = "header")
positive_cloned_clips_validation, durations = filter_audio_paths(['./datasets/cloned_samples/positive_validation'],min_length_secs=0.0, max_length_secs=max_duration_sec, duration_method = "header")

audio_samples_per_clip = 2*16000
for i in range(0,1):
    print("Augmenting clips with noise and room impulse response")

    positive_train_generator = augment_clips(positive_cloned_clips_train,
                        total_length=audio_samples_per_clip,
                        batch_size=config["augment_batch_size"],
                        background_clip_paths=[config["audioset_clips_dir"], config["fma_clips_dir"], config["fsd_clips_dir"]],
                        RIR_paths=rir_paths)
    positive_test_generator = augment_clips(positive_cloned_clips_test,
                        total_length=audio_samples_per_clip,
                        batch_size=config["augment_batch_size"],
                        background_clip_paths=[config["audioset_clips_dir"], config["fma_clips_dir"], config["fsd_clips_dir"]],
                        RIR_paths=rir_paths)
    positive_validation_generator = augment_clips(positive_cloned_clips_validation,
                        total_length=audio_samples_per_clip,
                        batch_size=config["augment_batch_size"],
                        background_clip_paths=[config["audioset_clips_dir"], config["fma_clips_dir"], config["fsd_clips_dir"]],
                        RIR_paths=rir_paths)



    dataset_name = "positive_cloned_samples"
    pos_dir_fname = "wakeword"
    augmented_positive_test_directory = os.path.join(config["features_output_dir"] + "/" + dataset_name + "/testing", pos_dir_fname)
    augmented_positive_validation_directory = os.path.join(config["features_output_dir"] + "/" + dataset_name + "/validation", pos_dir_fname)
    augmented_positive_train_directory = os.path.join(config["features_output_dir"] + "/" + dataset_name + "/training", pos_dir_fname)

    generator_outputs = [
                         [positive_train_generator, augmented_positive_train_directory, len(positive_cloned_clips_train)],
                         [positive_test_generator, augmented_positive_test_directory,len(positive_cloned_clips_test)],
                         [positive_validation_generator, augmented_positive_validation_directory, len(positive_cloned_clips_validation)],
                        ]


    for [generator, output_directory, n_total] in generator_outputs:

        output_directory = output_directory + '_batch_'+str(i)+'_mmap'
        print("Generating ", n_total, " features into ", output_directory)
        if not os.path.exists(output_directory):
            os.makedirs(output_directory, exist_ok=True)

        RaggedMmap.from_generator(out_dir=output_directory,sample_generator=clip_features_generator(generator),batch_size=4,verbose=True)



1999it [00:00, 242366.12it/s]
100%|██████████| 1999/1999 [00:00<00:00, 13330.62it/s]
1999it [00:00, 237109.07it/s]
100%|██████████| 1999/1999 [00:00<00:00, 13468.44it/s]
1939it [00:00, 249723.81it/s]
100%|██████████| 1939/1939 [00:00<00:00, 11139.74it/s]

Augmenting clips with noise and room impulse response
Generating  1999  features into  /home/lior/Custom-MicroWakeWord-Generator/training_features/positive_cloned_samples/training/wakeword_batch_0_mmap


0it [00:00, ?it/s]

Generating  1999  features into  /home/lior/Custom-MicroWakeWord-Generator/training_features/positive_cloned_samples/testing/wakeword_batch_0_mmap


0it [00:00, ?it/s]

Generating  1939  features into  /home/lior/Custom-MicroWakeWord-Generator/training_features/positive_cloned_samples/validation/wakeword_batch_0_mmap


0it [00:00, ?it/s]

In [33]:
###################################
# generate features based on COQUI'ed captured audio samples
###################################

max_duration_sec = 1.5
positive_pronounciation_clips_train, durations = filter_audio_paths(['./datasets/positive_more_pronunciations/alexa/positive_train/'],min_length_secs=0.0, max_length_secs=max_duration_sec, duration_method = "header")
positive_pronounciation_clips_test, durations = filter_audio_paths(['./datasets/positive_more_pronunciations/alexa/positive_test/'],min_length_secs=0.0, max_length_secs=max_duration_sec, duration_method = "header")
positive_pronounciation_clips_validation, durations = filter_audio_paths(['./datasets/positive_more_pronunciations/alexa/positive_validation'],min_length_secs=0.0, max_length_secs=max_duration_sec, duration_method = "header")




100000it [00:00, 273417.70it/s]
100%|██████████| 100000/100000 [00:07<00:00, 14230.11it/s]
1000it [00:00, 249067.93it/s]
100%|██████████| 1000/1000 [00:00<00:00, 14433.40it/s]
1000it [00:00, 243911.61it/s]
100%|██████████| 1000/1000 [00:00<00:00, 14516.18it/s]


In [39]:

print("Augmenting clips with noise and room impulse response")
audio_samples_per_clip = config["audio_samples_per_clip"]
positive_train_generator = augment_clips(positive_pronounciation_clips_train,
                    total_length=audio_samples_per_clip,
                    batch_size=config["augment_batch_size"],
                    background_clip_paths=[config["audioset_clips_dir"], config["fma_clips_dir"], config["fsd_clips_dir"]],
                    RIR_paths=rir_paths)
positive_test_generator = augment_clips(positive_pronounciation_clips_test,
                    total_length=audio_samples_per_clip,
                    batch_size=config["augment_batch_size"],
                    background_clip_paths=[config["audioset_clips_dir"], config["fma_clips_dir"], config["fsd_clips_dir"]],
                    RIR_paths=rir_paths)
positive_validation_generator = augment_clips(positive_pronounciation_clips_validation,
                    total_length=audio_samples_per_clip,
                    batch_size=config["augment_batch_size"],
                    background_clip_paths=[config["audioset_clips_dir"], config["fma_clips_dir"], config["fsd_clips_dir"]],
                    RIR_paths=rir_paths)



dataset_name = "positive_pronounciation_samples"
pos_dir_fname = "wakeword"
augmented_positive_test_directory = os.path.join(config["features_output_dir"] + "/" + dataset_name + "/testing", pos_dir_fname)
augmented_positive_validation_directory = os.path.join(config["features_output_dir"] + "/" + dataset_name + "/validation", pos_dir_fname)
augmented_positive_train_directory = os.path.join(config["features_output_dir"] + "/" + dataset_name + "/training", pos_dir_fname)

generator_outputs = [
                        [positive_train_generator, augmented_positive_train_directory, len(positive_pronounciation_clips_train)],
                        [positive_test_generator, augmented_positive_test_directory,len(positive_pronounciation_clips_test)],
                        [positive_validation_generator, augmented_positive_validation_directory, len(positive_pronounciation_clips_validation)],
                    ]




Augmenting clips with noise and room impulse response


In [40]:

for [generator, output_directory, n_total] in generator_outputs:

    output_directory = output_directory + '_batch_'+str(i)+'_mmap'
    print("Generating ", n_total, " features into ", output_directory)
    if not os.path.exists(output_directory):
        os.makedirs(output_directory, exist_ok=True)

    RaggedMmap.from_generator(out_dir=output_directory,sample_generator=clip_features_generator(generator),batch_size=100,verbose=True)


Generating  98329  features into  /home/lior/Custom-MicroWakeWord-Generator/training_features/positive_pronounciation_samples/training/wakeword_batch_0_mmap


0it [00:00, ?it/s]

Generating  985  features into  /home/lior/Custom-MicroWakeWord-Generator/training_features/positive_pronounciation_samples/testing/wakeword_batch_0_mmap


0it [00:00, ?it/s]

Generating  990  features into  /home/lior/Custom-MicroWakeWord-Generator/training_features/positive_pronounciation_samples/validation/wakeword_batch_0_mmap


0it [00:00, ?it/s]

In [ ]:
if running_on_colab:
    # copy training feastures dir to GDrive for training on the next notebook
    # backup generated samples
    !tar -cvf ./training_features_$(date +%Y%m%d_%H%M%S).tar ./training_features
    !cp ./training_features*.tar /content/drive/MyDrive/ColabNotebooks/VoiceAssistant/microWakeWord/

Bonus:

Visualizations of features: choose a sample ID to see its features from the sample itself *or* the version from the mmap which is augmented and is longer to account for initial condiftions inside the tflie based feature generator

In [ ]:
from mmap_ninja import RaggedMmap
generated_samples_features = RaggedMmap('./training_features/positive_tts_samples/training/wakeword_batch_0_mmap')
generated_samples_features = RaggedMmap('./training_features/positive_voice_samples/training/wakeword_batch_0_mmap')
sample_id = 0
print("There are ", len(generated_samples_features), " samples in the mmap dataset")
print("There are ", len(positive_captured_clips_train), " wav filenames in the original dataset")

In [ ]:
from mmap_ninja import RaggedMmap
import IPython.display as ipd
import matplotlib.pyplot as plt
import numpy as np
import librosa

mmap_fname = './training_features/positive_voice_samples/training/wakeword_batch_0_mmap'
fnames_list = positive_captured_clips_train
sample_id = 0

def show_feature(
        mmap_fname = './training_features/positive_voice_samples/training/wakeword_batch_0_mmap',
        fnames_list = positive_captured_clips_train,
        sample_id = 0):

    filename = fnames_list[sample_id]
    samples, sr = librosa.load(filename, sr=16000)
    # Play the audio
    audio = ipd.Audio(filename, rate=sr)
    ipd.display(audio)
    # plot the features from the wav itself
    samples = (samples*32767).astype(np.int16)
    filename = positive_captured_clips_train[sample_id]
    samples, sr = librosa.load(filename, sr=16000)
    samples = (samples*32767).astype(np.int16)
    plt.figure(figsize=(12, 6))
    plt.subplot(2, 1, 1)  # 2 rows, 1 column, second plot
    original_mfcc = generate_features_for_clip(samples).T
    plt.imshow(original_mfcc,  interpolation='nearest')
    plt.colorbar(format='%+2.0f')
    plt.title('original_mfcc')

    # plot the features from the mmap file
    generated_samples_features = RaggedMmap(mmap_fname)
    plt.subplot(2, 1, 2)  # 2 rows, 1 column, first plot
    uagmented_mfcc = generated_samples_features[sample_id].T
    plt.imshow(uagmented_mfcc)
    plt.colorbar(format='%+2.0f')
    plt.title('augmented_mfcc')
    # Create the second plot


    # Show the plots
    plt.tight_layout()
    plt.show()


#show_feature()
show_feature('./training_features/positive_tts_samples/training/wakeword_batch_0_mmap', positive_tts_clips_train, 0)